In [5]:
import pickle

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from keras.layers import Dense, Concatenate,Dropout, Input, Embedding, LSTM, Bidirectional,Flatten,SpatialDropout1D, CuDNNLSTM ,Conv1D,GlobalMaxPooling1D
from keras.models import Model
from keras.models import Sequential
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import re
from ckonlpy.tag import Twitter
from konlpy.tag import Mecab
from keras.utils import np_utils
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout,SpatialDropout1D
from sklearn.preprocessing import LabelBinarizer
import keras
import keras.preprocessing.text
from keras.preprocessing import sequence
from random import *
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional,SpatialDropout1D, MaxPooling1D
from keras.models import Sequential
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
import copy
from sklearn.metrics import classification_report
from keras.models import load_model
import keras.backend.tensorflow_backend as K
import tensorflow as tf

# In[2]:


# Random_state
rs_list = [1]
history_list = []
for rs in rs_list:
    print(rs)
    # 트레이닝 테스트 데이터 분리

    f1 = pd.read_excel("./data/emoji_final.xlsx")
    f1_x_list = f1['comments'].tolist()
    f1_y_list = f1['label'].tolist()

    f1_x_train, f1_x_test,f1_y_train,f1_y_test = train_test_split(f1_x_list,f1_y_list,test_size = 0.2,random_state=rs, stratify=f1_y_list)
    f1 = pd.DataFrame(list(zip(f1_x_train,f1_y_train)),columns=['comments','label'])

    # In[4]:

    f2 = pd.read_excel("./data/sub_neg_neg_df.xlsx")
    f2['label'] = 1
    f3 = pd.read_excel("./data/sub_pos_pos_df.xlsx")
    f3['label'] = 0


    # In[5]:


    f2 = f2[['comments','label']]
    f3 = f3[['comments','label']]


    # In[6]:


    f = pd.concat([f1,f2,f3],axis=0)


    # In[7]:


    f = f.reset_index(drop=True)
    f = f.drop_duplicates()

    # # 전체 전처리 

    # In[8]:


    # 형태소 분석기

    pos_tagger1 = Mecab()
    pos_tagger2 = Twitter()

    pos_tagger = pos_tagger1

    fidx = []
    t_result_list = []

    t_list = f['comments']

    # 형태소 분석이 어려운 샘플은 제외
    for idx,t in enumerate(t_list):
        try:

            t = re.sub('#', '', t)
            t = re.sub('\'', '', t)
            t = re.sub('\「', '', t)
            t = re.sub('\」', '', t)
            t = re.sub('\,', '', t)
            t = re.sub('#', '', t)
            t = re.sub('@', '', t)
            t = re.sub('\+', '', t)
            t = re.sub('[A-z]', '', t)
            t = re.sub('[0-9]', '', t)

            pos = list(pd.DataFrame(pos_tagger1.pos(t))[1])
            pos2 = list(pd.DataFrame(pos_tagger2.pos(t))[1])

            if len(pos) == 0 or len(pos2) == 0:
                print(str(idx)+"오류 제외")
            else :
                fidx.append(idx)
        except:
            print(str(idx)+"오류 제외")

    # 추가 전처리    
    f = f.iloc[fidx,:]


    # In[9]:


    BASE_CODE, CHOSUNG, JUNGSUNG = 44032, 588, 28

    # 초성 리스트. 00 ~ 18
    CHOSUNG_LIST = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

    # 중성 리스트. 00 ~ 20
    JUNGSUNG_LIST = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']

    # 종성 리스트. 00 ~ 27 + 1(1개 없음)
    JONGSUNG_LIST = [' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']


    def convert(test_keyword):
        split_keyword_list = list(test_keyword)
        #print(split_keyword_list)

        result = list()
        for keyword in split_keyword_list:
            # 한글 여부 check 후 분리
           # print(keyword)
            if re.match('.*[ㄱ-ㅎㅏ-ㅣ가-힣]+.*', keyword) is not None:
                try:
                    char_code = ord(keyword) - BASE_CODE
                    char1 = int(char_code / CHOSUNG)
                    if char1==0:  
                        result.append('')
                    result.append(CHOSUNG_LIST[char1])
                   # print('초성 : {}'.format(CHOSUNG_LIST[char1]))
                    char2 = int((char_code - (CHOSUNG * char1)) / JUNGSUNG)
                    if char2==0:
                        result.append('')
                    result.append(JUNGSUNG_LIST[char2])
                  #  print('중성 : {}'.format(JUNGSUNG_LIST[char2]))
                    char3 = int((char_code - (CHOSUNG * char1) - (JUNGSUNG * char2)))
                    if char3==0:
                        result.append('')
                    else:
                        result.append(JONGSUNG_LIST[char3])
                   # print('종성 : {}'.format(JONGSUNG_LIST[char3]))
                except:
                    result.append(keyword)
            else:
                result.append(keyword)
        # result
        return "".join(result)


    # # 통합모델

    # ## 형태소 분석 

    # In[10]:


    # Mecab 형태소 분석기 사용
    import os 
    os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
    os.environ["CUDA_VISIBLE_DEVICES"]="1"

    f1 = copy.deepcopy(f)

    t_list = f1['comments'].tolist()

    # Mecab 형태소 분석기 사용
    t_result_list = []

    # 형태소 분석결과 아무것도 안나오는 경우
    for idx,t in enumerate(t_list):
        try:
            t = re.sub('#', '', t)
            t = re.sub('\'', '', t)
            t = re.sub('\「', '', t)
            t = re.sub('\」', '', t)
            t = re.sub('\,', '', t)
            t = re.sub('#', '', t)
            t = re.sub('@', '', t)
            t = re.sub('\+', '', t)
            t = re.sub('[A-z]', '', t)
            t = re.sub('[0-9]', '', t)

            # 조사 제거
            pos = list(pd.DataFrame(pos_tagger.pos(t))[1])
            t = np.array(pos_tagger.morphs(t))
            t = t.tolist()
            t_result_list.append(" ".join(t))

        except:
            print(t)
            print(str(idx)+" 제외")

    f1['comments'] = t_result_list

    # label 교체
    label_list1 = f1['label']

    X_all1 = f1['comments'].tolist()
    X_all_list1 = X_all1
    Y_all_list1 = f1['label'].tolist()

    X_train1, X_test1,Y_train1,Y_test1 = train_test_split(X_all_list1,Y_all_list1,test_size = 0.2,random_state=rs, stratify=Y_all_list1)

    # 여기서 합쳐줘야댐 

    # convert string Label into 0,1 columns (3개의 컬럼)

    encoder = LabelBinarizer()

    X_train1 = pd.DataFrame(X_train1)
    Y_train1 = pd.DataFrame(Y_train1)
    all_train1 = pd.concat([X_train1,Y_train1],axis=1)
    all_train1.columns=['X','Y']


    X_train1 = all_train1['X'].tolist()
    Y_train1 = all_train1['Y'].tolist()
    Y_train1 = pd.DataFrame(encoder.fit_transform(Y_train1))
    Y_test1 = pd.DataFrame(encoder.fit_transform(Y_test1))

    # Binary Label인 경우에만

    Y_train1[1] = Y_train1[0]
    Y_train1[0] = -1*(Y_train1[0]-1)
    Y_test1[1] = Y_test1[0]
    Y_test1[0] = -1*(Y_test1[0]-1)

    max_slen1 = 200
    embedding_vecor_length1 = 128

    tk1 = keras.preprocessing.text.Tokenizer(lower=True, split=" ")
    tk1.fit_on_texts(list(X_train1))

    X_all1 = X_train1 + X_test1
    X_train1 = tk1.texts_to_sequences(X_train1)
    X_test1 = tk1.texts_to_sequences(X_test1)
    X_all1 = tk1.texts_to_sequences(X_all1)
    #Y_all_arr = np.array(Y_all_df)


    X_train1 = sequence.pad_sequences(X_train1, maxlen=max_slen1,padding='pre')
    X_test1 = sequence.pad_sequences(X_test1, maxlen=max_slen1,padding='pre')
    X_all1 = sequence.pad_sequences(X_all1, maxlen=max_slen1,padding='pre')

    nwords1 = X_train1.max() + 1

    print("reading data complete")


    # In[11]:


    X_train1.shape


    # ## 자모단위

    # In[12]:


    # 댓글 파일 로드

    f2 = copy.deepcopy(f)

    t_list = f2['comments'].tolist()
    t_result_list = []


    for t in t_list:
        try:
            t_result_list.append(t)
        except:
            print(str(idx)+" 제외")

    # # 추가 전처리    

    for i,s in enumerate(t_result_list):
        s = str(s)
        s = s.strip()
        temp_list = s.split(" ")

       # 특수문자 제거  
        s = re.sub('#', '', s)
        s = re.sub('\'', '', s)
        s = re.sub('\「', '', s)
        s = re.sub('\」', '', s)
        s = re.sub('\,', '', s)
        s = re.sub('#', '', s)
        s = re.sub('@', '', s)
        s = re.sub('\+', '', s)
        s = re.sub('[A-z]', '', s)
        s = re.sub('[0-9]', '', s)
        s = s.strip()

        t_result_list[i] = s

    f2['comments'] = t_result_list


    # label 2개로
    label_list2 = f2['label']

    max_slen2 = 500
    embedding_vecor_length2 = 128

    X_all2 = f2['comments']
    X_all_list2 = X_all2.tolist()

    for i, comment in enumerate(X_all_list2):
        temp = convert(comment)

        # 빈칸제거
        temp = temp.replace(" ","")
        temp2 = ""
        for t in temp:
            temp2 = temp2 + t +  " "
        temp2 = temp2[0:-1]
        X_all_list2[i] = temp2


    f2['comments'] = X_all_list2
    f2 = f2.reset_index(drop=True)



    X_all_list2 = f2['comments'].tolist()

    # LABEL을 데이터 프레임 형태로
    Y_all_list2 = f2['label'].tolist()
    Y_all_df2 = pd.DataFrame(Y_all_list2)

    # 숫자 매핑

    nwords2 = 320

    tk2 = keras.preprocessing.text.Tokenizer(num_words=nwords2,char_level=True,lower=False, split="")

    X_train2,X_test2,Y_train2,Y_test2 = train_test_split(X_all_list2,Y_all_list2,test_size = 0.2,random_state=rs,stratify=Y_all_list2)

    tk2.fit_on_texts(X_train2)

    X_train2 = tk2.texts_to_sequences(X_train2)
    X_test2 = tk2.texts_to_sequences(X_test2)
    X_all2 = tk2.texts_to_sequences(X_all2)


    # convert string Label into 0,1 columns (3개의 컬럼)
    encoder = LabelBinarizer()

    Y_all_df2 = pd.DataFrame(encoder.fit_transform(Y_train2))
    Y_all_df2[1] = Y_all_df2[0]
    Y_all_df2[0] = -1*(Y_all_df2[0]-1)
    Y_train2 = Y_all_df2

    Y_all_df2 = pd.DataFrame(encoder.fit_transform(Y_test2))
    Y_all_df2[1] = Y_all_df2[0]
    Y_all_df2[0] = -1*(Y_all_df2[0]-1)
    Y_test2 = Y_all_df2

    X_train2 = sequence.pad_sequences(X_train2, maxlen=max_slen2,padding='pre')
    X_test2 = sequence.pad_sequences(X_test2, maxlen=max_slen2,padding='pre')
    X_all2 = sequence.pad_sequences(X_all2, maxlen=max_slen2,padding='pre')
    nwords2 = X_train2.max() + 1
    print("reading data complete")


    # # 한글자단위

    # In[13]:


    # Mecab 형태소 분석기 사용

    f3 = copy.deepcopy(f)

    t_list = f3['comments'].tolist()

    # Mecab 형태소 분석기 사용
    t_result_list = []

    # 형태소 분석결과 아무것도 안나오는 경우
    for idx,t in enumerate(t_list):
        try:
            t = re.sub('#', '', t)
            t = re.sub('\'', '', t)
            t = re.sub('\「', '', t)
            t = re.sub('\」', '', t)
            t = re.sub('\,', '', t)
            t = re.sub('#', '', t)
            t = re.sub('@', '', t)
            t = re.sub('\+', '', t)
            t = re.sub('[A-z]', '', t)
            t = re.sub('[0-9]', '', t)
            t = t.strip()

            # 조사 제거
            t = t.replace(" ", "") 
            t = list(t)
            t_result_list.append(" ".join(t))

        except:
            print(str(idx)+" 제외")


    f3['comments'] = t_result_list

    # label 교체
    label_list3 = f3['label']

    X_all3 = f3['comments'].tolist()
    X_all_list3 = X_all3
    Y_all_list3 = f3['label'].tolist()

    X_train3, X_test3,Y_train3,Y_test3 = train_test_split(X_all_list3,Y_all_list3,test_size = 0.2,random_state=rs, stratify=Y_all_list3)


    # convert string Label into 0,1 columns (3개의 컬럼)

    encoder = LabelBinarizer()

    X_train3 = pd.DataFrame(X_train3)
    Y_train3 = pd.DataFrame(Y_train3)
    all_train3 = pd.concat([X_train3,Y_train3],axis=1)
    all_train3.columns=['X','Y']


    X_train3 = all_train3['X'].tolist()
    Y_train3 = all_train3['Y'].tolist()
    Y_train3 = pd.DataFrame(encoder.fit_transform(Y_train3))
    Y_test3 = pd.DataFrame(encoder.fit_transform(Y_test3))

    # Binary Label인 경우에만

    Y_train3[1] = Y_train3[0]
    Y_train3[0] = -1*(Y_train3[0]-1)
    Y_test3[1] = Y_test3[0]
    Y_test3[0] = -1*(Y_test3[0]-1)

    max_slen3 = 200
    embedding_vecor_length3 = 128

    tk3 = keras.preprocessing.text.Tokenizer(lower=True, split=" ")
    tk3.fit_on_texts(list(X_train3))

    X_all3 = X_train3 + X_test3
    X_train3 = tk3.texts_to_sequences(X_train3)
    X_test3 = tk3.texts_to_sequences(X_test3)
    X_all3 = tk3.texts_to_sequences(X_all3)
    #Y_all_arr = np.array(Y_all_df)


    X_train3 = sequence.pad_sequences(X_train3, maxlen=max_slen3,padding='pre')
    X_test3 = sequence.pad_sequences(X_test3, maxlen=max_slen3,padding='pre')
    X_all3 = sequence.pad_sequences(X_all3, maxlen=max_slen3,padding='pre')

    nwords3 = X_train3.max() + 1

    print("reading data complete")


    # # 형태소 분석 (Twitter)

    # In[14]:


    # Mecab 형태소 분석기 사용
    import os 
    os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
    os.environ["CUDA_VISIBLE_DEVICES"]="1"

    # Random_state

    f4 = copy.deepcopy(f)

    t_list = f4['comments'].tolist()

    # Mecab 형태소 분석기 사용
    t_result_list = []

    # 형태소 분석결과 아무것도 안나오는 경우
    for idx,t in enumerate(t_list):
        try:
            t = re.sub('#', '', t)
            t = re.sub('\'', '', t)
            t = re.sub('\「', '', t)
            t = re.sub('\」', '', t)
            t = re.sub('\,', '', t)
            t = re.sub('#', '', t)
            t = re.sub('@', '', t)
            t = re.sub('\+', '', t)
            t = re.sub('[A-z]', '', t)
            t = re.sub('[0-9]', '', t)


            # 조사 제거
            pos = list(pd.DataFrame(pos_tagger2.pos(t))[1])
            t = np.array(pos_tagger2.morphs(t))
            t = t.tolist()
            t_result_list.append(" ".join(t))

        except:
            print(t)
            print(str(idx)+" 제외")

    f4['comments'] = t_result_list

    # label 교체
    label_list4 = f4['label']

    X_all4 = f4['comments'].tolist()
    X_all_list4 = X_all4
    Y_all_list4 = f4['label'].tolist()

    X_train4, X_test4,Y_train4,Y_test4 = train_test_split(X_all_list4,Y_all_list4,test_size = 0.2,random_state=rs, stratify=Y_all_list4)


    # convert string Label into 0,1 columns (3개의 컬럼)

    encoder = LabelBinarizer()

    X_train4 = pd.DataFrame(X_train4)
    Y_train4 = pd.DataFrame(Y_train4)
    all_train4 = pd.concat([X_train4,Y_train4],axis=1)
    all_train4.columns=['X','Y']


    X_train4 = all_train4['X'].tolist()
    Y_train4 = all_train4['Y'].tolist()
    Y_train4 = pd.DataFrame(encoder.fit_transform(Y_train4))
    Y_test4 = pd.DataFrame(encoder.fit_transform(Y_test4))

    # Binary Label인 경우에만

    Y_train4[1] = Y_train4[0]
    Y_train4[0] = -1*(Y_train4[0]-1)
    Y_test4[1] = Y_test4[0]
    Y_test4[0] = -1*(Y_test4[0]-1)

    max_slen4 = 200
    embedding_vecor_length4 = 128

    tk4 = keras.preprocessing.text.Tokenizer(lower=True, split=" ")
    tk4.fit_on_texts(list(X_train4))

    X_all4 = X_train4 + X_test4
    X_train4 = tk4.texts_to_sequences(X_train4)
    X_test4 = tk4.texts_to_sequences(X_test4)
    X_all4 = tk4.texts_to_sequences(X_all4)
    #Y_all_arr = np.array(Y_all_df)


    X_train4 = sequence.pad_sequences(X_train4, maxlen=max_slen4,padding='pre')
    X_test4 = sequence.pad_sequences(X_test4, maxlen=max_slen4,padding='pre')
    X_all4 = sequence.pad_sequences(X_all4, maxlen=max_slen4,padding='pre')

    nwords4 = X_train4.max() + 1

    print("reading data complete")


    # In[15]:


    f_X_test = pd.Series(f1_x_test)
    f_Y_test = pd.Series(f1_y_test)


    # In[16]:


    idx_list = []

    # 형태소 분석이 어려운 샘플은 제외
    for idx,t in enumerate(f_X_test):
        try:
            t = re.sub('#', '', t)
            t = re.sub('\'', '', t)
            t = re.sub('\「', '', t)
            t = re.sub('\」', '', t)
            t = re.sub('\,', '', t)
            t = re.sub('#', '', t)
            t = re.sub('@', '', t)
            t = re.sub('\+', '', t)
            t = re.sub('[A-z]', '', t)
            t = re.sub('[0-9]', '', t)

            pos = list(pd.DataFrame(pos_tagger1.pos(t))[1])
            pos2 = list(pd.DataFrame(pos_tagger2.pos(t))[1])

            if len(pos) == 0 or len(pos2) == 0:
                print(str(idx)+"오류 제외")
            else :
                idx_list.append(idx)
        except:
            print(str(idx)+"오류 제외")

    f_X_test =  f_X_test[idx_list].tolist()
    f_Y_test =  f_Y_test[idx_list].tolist()


    # In[17]:


    # Mecab 형태소 분석기 사용
    t_result_list = []

    # 형태소 분석결과 아무것도 안나오는 경우
    for idx,t in enumerate(f_X_test):
        try:
            t = re.sub('#', '', t)
            t = re.sub('\'', '', t)
            t = re.sub('\「', '', t)
            t = re.sub('\」', '', t)
            t = re.sub('\,', '', t)
            t = re.sub('#', '', t)
            t = re.sub('@', '', t)
            t = re.sub('\+', '', t)
            t = re.sub('[A-z]', '', t)
            t = re.sub('[0-9]', '', t)

            # 조사 제거
            pos = list(pd.DataFrame(pos_tagger.pos(t))[1])
            t = np.array(pos_tagger.morphs(t))
            t = t.tolist()
            t_result_list.append(" ".join(t))

        except:
            print(t)
            print(str(idx)+" 제외")

    f_X_test1 = t_result_list


    # In[18]:


    t_result_list = []
    for i,s in enumerate(f_X_test):
        s = str(s)
        s = s.strip()
        temp_list = s.split(" ")

       # 특수문자 제거  
        s = re.sub('#', '', s)
        s = re.sub('\'', '', s)
        s = re.sub('\「', '', s)
        s = re.sub('\」', '', s)
        s = re.sub('\,', '', s)
        s = re.sub('#', '', s)
        s = re.sub('@', '', s)
        s = re.sub('\+', '', s)
        s = re.sub('[A-z]', '', s)
        s = re.sub('[0-9]', '', s)
        s = s.strip()

        t_result_list.append(s)

    X_all_list2 = t_result_list

    for i, comment in enumerate(X_all_list2):
        temp = convert(comment)

        # 빈칸제거
        temp = temp.replace(" ","")
        temp2 = ""
        for t in temp:
            temp2 = temp2 + t +  " "
        temp2 = temp2[0:-1]
        X_all_list2[i] = temp2

    f_X_test2 = X_all_list2


    # In[19]:


    # Mecab 형태소 분석기 사용
    t_result_list = []

    # 형태소 분석결과 아무것도 안나오는 경우
    for idx,t in enumerate(f_X_test):
        try:
            t = re.sub('#', '', t)
            t = re.sub('\'', '', t)
            t = re.sub('\「', '', t)
            t = re.sub('\」', '', t)
            t = re.sub('\,', '', t)
            t = re.sub('#', '', t)
            t = re.sub('@', '', t)
            t = re.sub('\+', '', t)
            t = re.sub('[A-z]', '', t)
            t = re.sub('[0-9]', '', t)
            t = t.strip()

            # 조사 제거
            t = t.replace(" ", "") 
            t = list(t)
            t_result_list.append(" ".join(t))

        except:
            print(str(idx)+" 제외")

    f_X_test3 = t_result_list


    # In[20]:


    # Mecab 형태소 분석기 사용
    t_result_list = []

    # 형태소 분석결과 아무것도 안나오는 경우
    for idx,t in enumerate(f_X_test):
        try:
            t = re.sub('#', '', t)
            t = re.sub('\'', '', t)
            t = re.sub('\「', '', t)
            t = re.sub('\」', '', t)
            t = re.sub('\,', '', t)
            t = re.sub('#', '', t)
            t = re.sub('@', '', t)
            t = re.sub('\+', '', t)
            t = re.sub('[A-z]', '', t)
            t = re.sub('[0-9]', '', t)

            # 조사 제거
            pos = list(pd.DataFrame(pos_tagger2.pos(t))[1])
            t = np.array(pos_tagger2.morphs(t))
            t = t.tolist()
            t_result_list.append(" ".join(t))        
        except:
            print(t)
            #print(str(idx)+" 제외")

    f_X_test4 = t_result_list


    # In[21]:


    f_X_test1 = tk1.texts_to_sequences(f_X_test1)
    f_X_test1 = sequence.pad_sequences(f_X_test1, maxlen=max_slen1,padding='pre')
    f_X_test2 = tk2.texts_to_sequences(f_X_test2)
    f_X_test2 = sequence.pad_sequences(f_X_test2, maxlen=max_slen2,padding='pre')
    f_X_test3 = tk3.texts_to_sequences(f_X_test3)
    f_X_test3 = sequence.pad_sequences(f_X_test3, maxlen=max_slen3,padding='pre')
    f_X_test4 = tk4.texts_to_sequences(f_X_test4)
    f_X_test4 = sequence.pad_sequences(f_X_test4, maxlen=max_slen4,padding='pre')


    # In[22]:


    nwords1 = X_train1.max() + 2
    nwords2 = X_train2.max() + 2
    nwords3 = X_train3.max() + 2
    nwords4 = X_train4.max() + 2


    # In[23]:




    # In[24]:


    class TestCallback(tf.keras.callbacks.Callback):
        def __init__(self, test_data):
            self.test_data = test_data

        def on_epoch_end(self, epoch, logs={}):
            x, y = self.test_data
            loss, acc = self.model.evaluate(x, y, verbose=0)
            print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))


    # In[25]:


    # Binary Label인 경우에만
    f_Y_test = pd.DataFrame(f_Y_test)
    f_Y_test[1] = f_Y_test[0]
    f_Y_test[0] = -1*(f_Y_test[0]-1)


    # In[26]:


    Y_all = pd.concat([Y_train1,Y_test1],axis=0)


    # In[27]:


    # Bi-LSTM (형태소, 문자단위)

    
    NUM_FILTERS = 64
    NUM_WORDS = 3 
    early_stopping = EarlyStopping(monitor='val_loss',patience=1,restore_best_weights=True)

    with K.tf.device('/gpu:0'):
        model1 = Sequential()
        model1.add(Embedding(nwords1, embedding_vecor_length1, input_length=max_slen1))
        model1.add(SpatialDropout1D(0.2))
        model1.add(Conv1D(filters=NUM_FILTERS, kernel_size=NUM_WORDS, activation="relu")) 
        model1.add(Dropout(0.2)) 
        model1.add(Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2)))    
        model1.add(Dropout(0.2))
        model1.add(Dense(64,activation='relu'))

        model2 = Sequential()
        model2.add(Embedding(nwords2, embedding_vecor_length2, input_length=max_slen2))
        model2.add(SpatialDropout1D(0.2)) 
        model2.add(Conv1D(filters=NUM_FILTERS, kernel_size=NUM_WORDS, activation="relu")) 
        model2.add(Dropout(0.2)) 
        model2.add(Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2)))
        model2.add(Dropout(0.2))
        model2.add(Dense(64, activation='relu'))

        model3 = Sequential()
        model3.add(Embedding(nwords3, embedding_vecor_length3, input_length=max_slen3))
        model3.add(SpatialDropout1D(0.2)) 
        model3.add(Conv1D(filters=NUM_FILTERS, kernel_size=NUM_WORDS, activation="relu")) 
        model3.add(Dropout(0.2)) 
        model3.add(Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2)))
        model3.add(Dropout(0.2))
        model3.add(Dense(64, activation='relu'))

        model4 = Sequential()
        model4.add(Embedding(nwords4, embedding_vecor_length4, input_length=max_slen4))
        model4.add(SpatialDropout1D(0.2)) 
        model4.add(Conv1D(filters=NUM_FILTERS, kernel_size=NUM_WORDS, activation="relu")) 
        model4.add(Dropout(0.2)) 
        model4.add(Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2)))
        model4.add(Dropout(0.2))
        model4.add(Dense(64, activation='relu'))

        model_input1 = Input((max_slen1,))
        model_input2 = Input((max_slen2,))
        model_input3 = Input((max_slen3,))
        model_input4 = Input((max_slen4,))

        out1 = model1(model_input1)   
        out2 = model2(model_input2)
        out3 = model3(model_input3)
        out4 = model4(model_input4)

        out = Concatenate()([out1,out2,out3,out4])
        out = Dense(2, activation='softmax')(out)

        model = Model([model_input1,model_input2,model_input3,model_input4],out)
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        #history= model.fit([X_train1,X_train2,X_train3,X_train4],Y_train1, epochs=20, batch_size=4096,validation_data=([X_test1,X_test2,X_test3,X_test4],Y_test1),shuffle=False,callbacks=[early_stopping,TestCallback(([f_X_test1,f_X_test2,f_X_test3,f_X_test4], f_Y_test))])
        #history= model.fit([X_train1,X_train2,X_train3,X_train4],Y_train1, epochs=20, batch_size=4096,validation_data=([f_X_test1,f_X_test2,f_X_test3,f_X_test4], f_Y_test),shuffle=False,callbacks=[early_stopping])
        history= model.fit([X_all1,X_all2,X_all3,X_all4],Y_all, epochs=20, batch_size=4096,validation_data=([f_X_test1,f_X_test2,f_X_test3,f_X_test4], f_Y_test),shuffle=False,callbacks=[early_stopping])
    
    history_list.append(history.history)
    model.save('senti_model_'+str(rs) +'.h5')
    # saving
    with open('tk1_' + str(rs) + '.pickle', 'wb') as handle:
        pickle.dump(tk1, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('tk2_' + str(rs) + '.pickle', 'wb') as handle:
        pickle.dump(tk2, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('tk3_' + str(rs) + '.pickle', 'wb') as handle:
        pickle.dump(tk3, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('tk4_' + str(rs) + '.pickle', 'wb') as handle:
        pickle.dump(tk4, handle, protocol=pickle.HIGHEST_PROTOCOL)


2


/usr/local/lib/python3.6/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


113오류 제외
1377오류 제외
2623오류 제외
2972오류 제외
5965오류 제외
10049오류 제외
10249오류 제외
12258오류 제외
12409오류 제외
14122오류 제외
16189오류 제외
17618오류 제외
18487오류 제외
19808오류 제외
21931오류 제외
23680오류 제외
25685오류 제외
27013오류 제외
29914오류 제외
32050오류 제외
32144오류 제외
32222오류 제외
32652오류 제외
32695오류 제외
35033오류 제외
44839오류 제외
92958오류 제외
137531오류 제외
reading data complete
reading data complete
reading data complete
reading data complete
1344오류 제외
5569오류 제외
8770오류 제외
Train on 181543 samples, validate on 9991 samples
Epoch 1/20
181543/181543 [==============================] - 2778s 15ms/step - loss: 0.3555 - accuracy: 0.8486 - val_loss: 0.4898 - val_accuracy: 0.7840
Epoch 2/20
181543/181543 [==============================] - 2768s 15ms/step - loss: 0.0981 - accuracy: 0.9595 - val_loss: 0.4646 - val_accuracy: 0.8025
Epoch 3/20
181543/181543 [==============================] - 2759s 15ms/step - loss: 0.0691 - accuracy: 0.9727 - val_loss: 0.5098 - val_accuracy: 0.7964
3


/usr/local/lib/python3.6/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


18오류 제외
4399오류 제외
4569오류 제외
6751오류 제외
11454오류 제외
12755오류 제외
13887오류 제외
14165오류 제외
18358오류 제외
19787오류 제외
21704오류 제외
22050오류 제외
25146오류 제외
25944오류 제외
26535오류 제외
26858오류 제외
27097오류 제외
27974오류 제외
29138오류 제외
29589오류 제외
31712오류 제외
37307오류 제외
38060오류 제외
38930오류 제외
44839오류 제외
92958오류 제외
137531오류 제외
reading data complete
reading data complete
reading data complete
reading data complete
1681오류 제외
3204오류 제외
6346오류 제외
Train on 181544 samples, validate on 9991 samples
Epoch 1/20
126976/181544 [===================>..........] - ETA: 13:53 - loss: 0.4365 - accuracy: 0.7929

In [6]:
with open('tk1_' + str(rs) + '.pickle', 'wb') as handle:
    pickle.dump(tk1, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('tk2_' + str(rs) + '.pickle', 'wb') as handle:
    pickle.dump(tk2, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('tk3_' + str(rs) + '.pickle', 'wb') as handle:
    pickle.dump(tk3, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('tk4_' + str(rs) + '.pickle', 'wb') as handle:
    pickle.dump(tk4, handle, protocol=pickle.HIGHEST_PROTOCOL)